<a href="https://colab.research.google.com/github/RyanSmoak/MNIST-from-scratch/blob/main/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MNIST: Handwritten digit recognition

In this cookbook, we will work on building a CNN from scratch to identify and recognise handwritten digits (the common MNIST problem). This mini project is limited to only recognition and not other computer vision tasks. Some key notes before starting:
1. All layers in this cookbook will be implemented from scratch. That is:
  - Convolutional layer
  - Fully connneted layer (dense layer)
  - Pooling layer
  - Flattening layer
2. Activation functions (ReLU/PReLU/Softmax) and loss functions (cross-entropy) will be implemented from scratch.
3. The optimization technique used (SGD + Nestrov Momentum) will be implemented from scractch.
4. Before each code cell, there will a markdown cell that includes a detailed explanation of what the code does. Some (NOT all) will include a brief mathematical explanation.

Also be aware that the code in the code cells are just skeletons. They have not been debugged or modified for batch processing or broadcasting. They are just implementations to explain the content. The correct and modified code exists in separate files on GitHub

That being said, we can get started.

##Importing dependancies

Just like any project, we have to import some quick dependancies/libraries. These dependancies include:
1. numpy - Honestly, if you know anything about AI/ML, you should not be shocked that this library is here. It's everywhere in AI/ML, because it allows to manipulate the most basic data in AI, numbers.
2.  signal - Usually used in signal processing, but this library has some useful functions for our use. These functions are the cross-correlation and convolution functions. To be disccused later.

In [ ]:
import numpy as np
from scipy import signal

##**Convolutional (Conv2D) Layer**

### **Forward Prop**



The first thing is of course to work write our convoluation layer since it is the most important thing. Some quick explanation of what convolution/convLayer is, first. I will be detailed but not go too deep into it. We'll start with the forward pass part of the network before working on the backprop.

Firstly, the difference between a convulational layer and a standard fully connected (dense) layer. It's quite simple really. A convolutional layer simply preserves the spatial structure of it's input. This is really important since we want to preserve key parts of an image (which is what we deal in with CNNs).
>Example: *A $32 x 32 x 3$ image would be strecthed to a $3074x1$ matrix for a fully connected layer. In a conv layer, it would remain as is.*

The actual convolution process is by convolving the image with a filter (AKA a kernel, receptive field) of equal depth i.e *slide the filter matrix over the image and compute it's dot product.* The result of this operation is called an activation map.

The convolution operation itself **looks** as follows:
> $$Y_i = B_i \quad+\quad \sum_{j=1}^{n}X_j *K_{ij},\qquad i = 1...d$$
Where $Y$ is the output, $B$ is the bias, $X$ is the input matrix, $K$ is the kernel/filter/receptive field. The depth of the output is same as that of the input structure.
>>Also, note that the asterik ($*$) is not element-wise multiplication of the matrices. It is the convolution process which is a dot product of a part of the input matrix and the filter.

Above is the basic convolution process. There are 2 more things we need to deal with before this becomes a fully functioning convolutional layer. We need to think about:
1. Stride (S): This is the number of steps the filter takes while sliding across the input structure
```
h, w = 6
stride = 2
#Calculate the top-left corner of the current window
h_start = h * stride
w_start = w * stride
F_h, F_w = 2
#Extract the input slice
input_slice = input[:, h_start:h_start + F_h, w_start:w_start + F_w]
```
2. Padding (P): This is a layer added all around the input structure at the boarders to maintain spatial integrity and prevent data loss along the edges. We will use zero padding for this project.
```
D_1, H_1, W_1 = input.shape
#create a padded array of zeros
padded_input = np.zeros((D_1, H_1 + 2 * self.padding, W_1 + 2 * self.padding))
#copy the input structure into the centre of the padded array
padded_input[:, self.padding:self.padding + H_1, self.padding:self.padding + W_1] = input
```

Taking into account our new hyperparameters, we have a new formula for the getting the output matrix volume:
$$\left(\frac{N-F+2P}{S}\right) + 1$$
> Remember, the depth is not altered. The output depth is the same as the structure depth.



---


#### **The filters**

We need to talk about this filters a little cause it may not be as obvious what they are. The simplest way to explain them is as weights. Actually, that's exactly what they are. Feature weights. In standard ML, we are used to each input feature being associated with a certain weight. Well, that's what we're doing but now each input feature ($d$-th depth of an input image) is associated with weight matrix (filter).

Just like any ML/DL model, we need to initialise the weights beforehand. That is, before all the work of building up the model, we need to have some random weights to start with so the model can learn and update them.

How do we do this? The go-to method for small networks is just random small random numbers. This is good for small networks since learning isn't deep, so while using backprop, our gradients are less likely to reach 0 (they may tend to 0, but not get to 0). For larger, more sophisticated networks, we use better methods.

In this cookbook, we will use one of these better methods: He weight initialization. Why? Just cause we can😁. The basic idea of He initialization is initializing weights with values drawn from a normal distribution, but with a variance that is scales according to the number of input neurons for each weight. Of course it has a formula and of course I'll add it here cause it's a really fun one:
> $$w \approx \mathcal{N} \left(0, \frac{2}{n_{in}}\right)$$
> where:
- $\mathcal{N}$ is the normal (Gaussian) distribution with mean $0$ and variance $ \frac{2}{n_{in}}$
- $n_{in}$ is the number of input neurons to the current layer.

The code for this generally looks like (in python):
```
input_depth, depth = 1
filter_size = 3
num_filters = 32

fan_in = input_depth * filter_size * filter_size
filter_shape = (depth, input_depth, filter_size, filter_size)
filters = np.random.randn(num_filters, input_depth, filter_size, filter_size)/np.sqrt(fan_in/2)
biases = np.zeros(num_filters)
```

I want to be fancy so, obviously I'd use this. There's a much simpler method known as Xavier initialization. Why didn't we use the easier one? Because the activation function we're using is PReLU, which would end up blowing up the gradients in the negative realm. We'll discuss this a bit deeper when we implement the activation function.

#### **A quick BTW**

I feel as if I should mention this early. There are MULTIPLE types of convolution layers. Some examples are:
1. **Conv1D** - ConvLayers that work with 1-D data like time series or text. Majorly used in sequential data analysis and audio processing.
2. **Conv2D** - ConvLayers that work with 2-D data like images. Input data usually 2 dimensions (height and width) with an extra 'dimension' (like channels). Used in image recognition and segmentation. This is what we'll be creating.
3. **Conv3D** - ConvLayers that work with 3-D data. Input data usually has 3 dimensions (height, width, and depth) and an extra 'dimension' (like channels). Used in volumetric data processing and time-series data with spatial dimenesions (video processing).
4. **Transposed convolution (Deconvolution)** - Used for upsampling the input. Works in the opposite manner as a normal convLayer by increasing the spatial dimensions of the input. Used in Generative Adversial Networks (GAN) and autoencoders.

And so many more...

We're building a Conv2D layer since we're working with images.

---

#### **Forward Prop Summary**

At this point, we have built the forward pass of our Conv Layer. A quick summary of what is going on there:

> Even though it wasn't the first thing we talked about, the first thing is to  initialize our filters (weights) using He initialization. This step also includes initiliazing various parameters such as:
- input shape
- number of filters
- stride
- padding

> After initialization, we've padded our input using zero padding and set a stride for our filter.

> Calculate the output shape volume using: $\left(\frac{N - F+2P}{S}\right)+1$

> We initialize the output matrix with the calculated size

> Perform the convolution process on the input structure to get and copy the results to the output matrix.

###**BackProp**

The back pass, known as backprop isn't as complicated as most people think it is. Once you understand the fundamental concepts of backprop in a FC (dense) layer, you'll easily get for any other sort of layer. All we need to do for the backprop in a CNN is calculate the gradient of the loss w.r.t the filters, bias and input. Like I said, in this cookbook I will not be going too deep so we won't discuss loss functions. I'll keep it basic.

For this cookbook, I'll write the formulas and give ONLY an intuition of why the work.

Some notation first:
- $E$ is the loss function. In our case, assume binary cross-entropy as our loss function
- $Y_i$ is the output produced by the forward pass at any given iteration
- $∂E$ is the error signal *i.e* the gradient of the loss function w.r.t $Y_i$
- $K_{rot}$ is the filter rotated $180°$ about its centre
- $K_{ij}$ is a cell of the filter at any given iteration
- $B$ is the bias associated with forward pass at any iteration
- $X_j = X_{region}[i,j]$ is the $K$ x $K$ patch of the input that the filter was applied to produce $Y[f,i,j]$
- $conv$  is the convolution process

1. Gradient w.r.t the filter
> $dW = convolve(Input, dOutput)$
>$$\frac{∂E}{∂K_{ij}} = \sum_i \frac{∂E}{∂Y_i}\quad\cdot\quad X_j$$
>> In the backward pass, the gradient of the output w.r.t the filter is the 'reverse' of this sliding that is done in the forward pass. We convolve the error signal $dOutput$ $(∂E)$ with the input to compute $dW$.
```
  for f in range(self.num_filters):
      for i in range(out_h):
          for j in range(out_w):
              region = input_padded[:,
                                    i * self.stride:i * self.stride + self.filter_size,
                                    j * self.stride:j * self.stride + self.filter_size]
              dE_dK[f] += region * dE_dY[f, i, j]
      dE_db[f] = np.sum(dE_dY[f])  # Sum over all positions for the bias gradient
```

2. Gradient w.r.t the bias
> $dB = \sum dOutput$
>$$\frac{∂E}{∂B_i} = \sum \frac{∂E}{∂Y_i} $$
>> In the forward pass, the bias ($b$) is added to every output at any iteration, which means it directly contributes to the output. Since $b$ is added uniformly to all output positions, the gradient of the loss w.r.t the bias is the sum of all elements in the gradient of the loss w.r.t the output.
```
df_db = output_gradient
```
Since the gradient w.r.t the bias is the same as the output gradient, it can be passed as a parameter to the backprop function from the next layer.

3. Gradient w.r.t the input
>$dInput = reverse\_convolve(dOuput, Filters)$
>$$\frac{∂E}{∂X_j} = \sum_{i=i}^{n}  conv\left(\frac{∂E}{∂Y_i}, K_{rot}\right) $$
>> The input can only contribute to the output through the filter $K_{ij}$. We can use this relatinship to compute $\frac{∂E}{∂X_j}$. The gradient is gotten by fully convolving our the filter $K_{rot}$ with tthe output gradient.
>>> **NOTE:** The convolution here is full and not valid, like it was in the forward pass. This is because the gradient needs to cover all positions of the input including those that may have been padded.  
```
for i in range(out_h):
  for j in range(out_w):
      for f in range(self.num_filters):
          region = dE_dX[:,
                          i * self.stride:i * self.stride + self.filter_size,
                          j * self.stride:j * self.stride + self.filter_size]
          region += self.filters[f] * dE_dY[f, i, j]
```

Like I said, the markdowns will be detailed but I will not go too deep into the theory. For this case, I will not be including the derivation of these formulas in this cookbook (even though, they are very fun and interesting). Instead, we will use the intuition I've given above and the simplified formulas and implement them in code.

After that, we can use another algorithm to update the weights. In this case, we'll use Adam's. To be discussed later.


###Full summary

At this point, we have knowledge to develop a algortihm for a basic convolutional layer. First, a summary of what the conv layer is meant to do.

>Accepts a volume of size $W_1*H_1*D_1$

>Requires 4 hypermaters
- Number of filters, $K$
- Spatial extent of the filter, $F$
- Stride, $S$
- Amount of zero padding, $P$

>Produce a volume of size $W_2*H_2*D_2$ where
- $W_2 = \left(\frac{W_1 - F+2P}{S}\right)+1$
- $H_2 = \left(\frac{H_1 - F+2P}{S}\right)+1$
- $D_2 = K$

>> **NOTE:** The depth is maintained. The width and height are computed equally by symmetry

>With parameter sharing, it introduces $F.F.D_1$ weights per filter for a total of $(F.F.D_1).K$ weights and $K$ biases.

> In he output's volume, the $d$-th depth slice is the result of performing a *valid* convoluton of the $d$-th filter over the input volume with a stride $S$ and then offset by the $d$-th bias.

>> **NOTE:** The word valid has been italicised since it is one of 2 types of convolutions:
1. Valid convolution - Where the kernel starts and stops at the boarder of the input structure
2. Full convolution - The kernel starts multiplication as soon as the intersection between the filter and the input structure is not 0 i.e *there's at least one cell in the filter matrix that is inline with the input.* This type of convolution leads to a larger output, which means it is more computationally expensive.

---


### Actual Conv2D Layer

Quick note: Wherever the word tensor has been used, just know that is a numpy matrix. I've called them tensors cause it's familiar to the AI engineers who use TensorFlow.

In [ ]:
class Conv2DLayer:
  '''
    This class initializes the convolutional layer, the most basic and important layer in our CNN.
  '''
  def __init__(self, input_shape, filter_shape, stride, padding):
    '''
      This function initializes the convolutional layer.
      Inputs:
        - input_shape: The shape of the input structure (depth, height, width)
        - filter_shape: The shape of the filter (num_filters, depth, height, width)
        - stride: The number of steps the filter takes at each iteration
        - padding: The amount of zero padding to be added to the input structure
    '''
    self.input_shape = input_shape
    self.input_depth = input_shape[0]

    self.num_filters = filter_shape[0]
    self.filter_size = filter_shape[2]
    self.stride = stride
    self.padding = padding

    #self.output_shape = (depth, input_height - filter_size + 1, input_width - filter_size + 1)
    fan_in = input_shape[0] * filter_shape[2] * filter_shape[2]
    self.filter_shape = filter_shape
    self.filters = np.random.randn(filter_shape[0], input_shape[0], filter_shape[2], filter_shape[2])/np.sqrt(fan_in/2)
    self.biases = np.zeros(filter_shape[0])

  def zero_pad(self, input):
    '''
    This function pads the input with zeros to a certain degree.
    Input:
      - input: 2D array (H_1 x W_1)
    Output:
      - padded_input: 2D  padded array (H_1+padding X W_2+padding)
    '''

    D_1, H_1, W_1 = input.shape
    #create a padded array of zeros
    padded_input = np.zeros((D_1, H_1 + 2 * self.padding, W_1 + 2 * self.padding))
    #copy the input structure into the centre of the padded array
    padded_input[:, self.padding:self.padding + H_1, self.padding:self.padding + W_1] = input

    return padded_input

  def stride(self, input, h, w, filter_size):
    '''
    This function is meant to slide the filter along the structure volume a certain number of steps a
    at each iteration.
    Inputs:
      - h: The height of the structure
      - w: The width of the structure
      - filter_size: The width and/or the height of the filter
    Output:
      - The input slice at a given iteration
    '''
    stride = self. stride
     # Calculate the top-left corner of the current window
    h_start = h * stride
    w_start = w * stride

    F_h, F_w = filter_size
    # Extract the input slice
    input_slice = input[:, h_start:h_start + F_h, w_start:w_start + F_w]

  def forward_pass(self):
    '''
    This function does the actual convolution process that we described ealier in this cookbook.
    '''
    input = self.input_shape
    stride = self.stride
    padding = self.padding
    filter_size = self.filter_size
    filters = self.filters
    biases = self.biases

    (C_in, H_in, W_in) = self.input_shape
    (num_filters, C_in_filter, F_h, F_w) = self.filter_shape

    H_out = (H_in - F_h + 2*padding) // stride + 1
    W_out = (W_in - F_w + 2*padding) // stride + 1

    # Initialize the output tensor
    output = np.zeros((num_filters, H_out, W_out))

    padded_input = self.zero_pad(input)

    # Perform convolution
    for n in range(num_filters):  # Loop over each filter
        for h in range(0, H_out):  # Loop over output height
            for w in range(0, W_out):  # Loop over output width
                input_slice = stride(padded_input, h, w, filter_size)

                # Perform dot product
                output[n, h, w] = np.sum(input_slice * filters[n]) + biases[n]

    return output

  def backward_pass(self, output_gradient, optimizer):
    '''
    This function performs the backward pass to the network that will
    '''
    filters_gradient= np.zeros(self.filter_shape)
    input_gradient = np.zeros(self.input_shape)
    bias_gradient = np.sum(output_gradient, axis=(1, 2)).reshape(self.biases.shape)

    #compute the gradients
    for i in range (self.num_filters):
      for j in range (self. input_shape[0]):
        filters_gradient[i, j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")
        input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], "full")

    self.filters, self.biases = optimizer.update(
        self.filters, self.biases, filters_gradient, bias_gradient
    )

    return filters_gradient, input_gradient, bias_gradient

#### Test the conv layer

We need to know and see if what we're doing actually makes sense and works. To do that, we need to be sure that our layer is creating and initilazing the filters.

To do this, we assume:
1. An input of shape (1,32,32)
2. We want to learn 32 features so assume 32 filters
3. Filter matrix of shape (3,3)

The other parameters don't really matter for us right now.

In [ ]:
#testing the Conv2DLayer class to see if our layer is actually working
input_shape = (1, 32, 32)
conv_layer = Conv2DLayer(num_filters=32, filter_size=3, stride=1, padding=1, input_shape=input_shape, depth=1)

print(f"Filters shape: {conv_layer.filter_shape}")
print(f"Filters: {conv_layer.filters}")
print(f"Biases: {conv_layer.biases.shape}")
print(f"Biases: {conv_layer.biases}")

Filters shape: (1, 1, 3, 3)
Filters: [[[[-0.77086145 -1.12572475 -0.23769746]
   [ 0.30934366 -0.06627308 -0.23660901]
   [ 0.35943154 -0.11117635  0.10582584]]]


 [[[ 0.38661059 -0.16423421  0.10269236]
   [ 0.64298742 -0.07480266 -0.4915935 ]
   [-0.54112349  0.57341941 -0.33600652]]]


 [[[ 0.09249641  0.14359097  0.10823315]
   [-0.11697796 -0.57625676 -0.11557431]
   [-0.18479033  0.21788764 -0.76555109]]]


 [[[-0.57658894 -0.61601796 -0.18299021]
   [-0.02229058  0.52362172  0.04523191]
   [ 0.35868255  0.26983439  0.62550827]]]


 [[[ 0.53095718  0.15639279  0.18376651]
   [ 0.11092405 -0.07730636 -0.43651223]
   [-0.17792025  0.66230161 -0.5030989 ]]]


 [[[ 0.48227481 -0.79866884  0.78630629]
   [-0.31932405 -0.4224309  -0.77922378]
   [ 0.49383822 -0.46682396 -0.2093662 ]]]


 [[[-0.72718066 -0.28478991 -0.2664904 ]
   [ 0.27084903  0.07652734 -0.04773216]
   [-0.62402334  0.05720462  0.09731497]]]


 [[[ 0.68417628 -0.50793259 -0.28599992]
   [ 0.19186677 -0.61705473 -0.04

Weights and biases are being initialized correctly with the correct shape

##**Optimization**

Now at this point of our existence, we have fully created and written our convolutional layer. Everything in that layer has implemented acccording to our theory that we've been cooking up in the markdowns. We've done all the thing that are needed for conv layer: padding, setting our stride, forward pass and backward pass. At the end of the backpass, we have our gradients.

What do we do with these gradients now? Well, so as to learn, the network has to update the weights to reduce the error loss and it has to do this iteratively. The network does this through something called optimization.

If you want to imagine how this works, think of it as walking down a valley and you want to get to the minimum point. Imagine our loss function giving us some curvature where the global minima is a point where $J(Θ) ≈ 0$ such that $J$ is our loss function. With our weights, we would like to take small steps in the steepest descent/ascent to get to the minima.

To do this updates/ take these steps, we use an optimization algorithm. There are so many algorithms to do this, such as:
1. Stochastic Gradient Descent (SGD)
2. SGD + Momentum
3. AdaGrad
4. DeGrad
5. RSMprop
6. Adaptive Movement eStimation (Adams)

In this project, we'll be implementing SGD with momentum. Why? Initially, I had planned on doing Adams and quickly realised how much explaining that would require. SGD is fairly simple. A small adjustment would give us SGD with momentum. Also our network is very shallow. So, let's get to it.

####Stochastic Gradient Descent (SGD)

We'll start here and build up to adding some momentum and what that even means. Like I had said earlier, to optimize an algorithm/model/NN, we walk down the path of steepest descent/ascent till we get to some global minima. This is called 'gradient descent'
> You may see some across the term 'the critical point' denoted with a $Θ^*$. Be aware that this doesn't mean the global minima. It is a candidate point that can be considered for both a local and global minima but has not been proven to be either.

SGD **is** gradient descent with a small alteration. Instead of taking the whole dataset to learn, it uses just a small subset of it. It's the most basic algorithm but has some problems:
1. What if the loss changes quickly in some dimension and slow in other dimensions?
- Well, SGD will end up progressing very fast in some directions and very slowly in some directions leading to a very zig-zag, jittery pattern that is very bad for learning.
> When this occurs, the loss function is said to a high condition. This means that the ratio of the largest to the smallest eigenvalue of the Hesian matrix is large. The best way to think of this is using a taco shell, where one side is steep (high curvature), and the other remains flat (low curvature); (i.e anisotropic).
>> Just a quick btw. A hesian matrix is a sqaure matrix of second-order partial derivatives of a scala-valued function that describes the curvature of a function with many variables.

2.  What if the loss a local minima or saddle point?
- Well, the zero gradient at this point will make SGD get stuck and assuming it's gotten to the global minima.
3. SGD takes random minibatches of data from the test data. This can lead to noise.

We've talked about SGD's problems but what even does our SGD look like, mathematically. Well, the formula for it is:
$$W_{t+1} := W_t - α⋅∇f(W_t)$$
where:
- $W_{t+1}$ is the new weight
- $W_t$ is the value of the current weight
- $α$ is the learning rate
- $∇$ is the result of the gradient computation
- $f(W_t)$ is our heuristic function

When I said that this is the most basic and simple optimization algorithm, I wasn't joking. It really is. The equation above is a simple update rule that allows us to update our weights (learn). But this is prome to all the issues I mentioned earlier.


#### SGD + Momentum
To solve the issues I mentioned above, we add momentum to our weight to build up "velocity" as a running mean. How does this help? Well, this way anytime we take a step we also have this momentum. The best analogy to use here is think of a ball that is rolling down a hill. This ball will remain in motion until it reaches somewhere flat. The only thing that would keep it moving is momentum till it gets to another descent where it will roll even more. That's the intution behind it.

The real reasoning: as the point moves down towards a critical point, it builds up a momentum which is the average of the gradients as it proceeds. This would help it combat the issue of zero-gradients at critical points as steps will be taken using the velocity and the gradient at that point. There are 2 types of Momentum:
1. Standard momentum: We take a step using the gradient and then add the velocity from that point
2. Nestrov momentum: We take a step using the velocity, find the gradient at the new point, add them, then take a step from the original.

For this project, we'll be implementing Nestrov's momentum, since it's always the better choice, in my opinion. But first, let's see the formulation for standard momentum and build off from it.

Standard momentum:
> $$V_{t+1} = ρV_t + ∇f(X_t) $$
$$X_{t+1} = X_t - αV_{t+1}$$
> where:
- $ρ$ is the momentum coefficient that controls the influence of past gradients (usually a high number like $0.9$ or $0.99$)
- $α$ is obviously the learning rate
- $∇f(x_t)$ is the gradient from the current point
- $V_t$ is the velocity at a certain point
>> To get some intuition about what is going on here: the first formula (velocity formula) computes the 'lookahead' position, where the gradient will end up after a step. The second part of the formula is the update rule that uses the velocity to update the weight.

From this, we can build up to Nestrov's momentum. The only change we'll make to the formula above is that in Nestrov's momentum, we take a step first using the velocity and then calculate the gradient from there. The forumla:
> $$V_{t+1} = ρV_t - α∇f(x_t + ρV_t)$$
$$X_{t+1} = X_t + V_{t+1}$$
The forumla is correct, but we often want to update terms in terms of $x_t$, $∇f(x_t)$. So we can rewrite our formula a bit so it's easier to implement in code:
$$V_{t+1} = ρV_t - α∇f({x^̃ }_t)$$
$${x^̃ }_{t+1} = \quad {x^̃ }_t - ρv_t + (1+ρ)v_{t+1} \quad= {x^̃ } + v_{t+1} + \rho(v_{t+1} - v_t)$$
where:
- ${x^̃}_t$ is the 'lookahead'; the estimated point where gradients will be calculated
- Ealier definitions of $\rho$, $α$ and $V_t$ are retained
>> The second formulation is a much easier thing to implement compared to the first one. In some texts, you may find it called 'Nestrov's Accelerated Momentum' but it's more or less the same thing but with more refinement. The refinement is brought by addding that $ρ(V_{t+1} - V_t)$ which provides a higher-order adjustment. Why?
>> - Smoothening oscillations hence reducing overshooting
>> - Adapting to change in direction of the gradient
>> - Accleration in flat regions


Let's see in code.

In [ ]:
import numpy as np

class SGD_NAG:
    def __init__(self, learning_rate, momentum):
        '''
        Always have an __init__ function in your class, seriously, do it.
        Inputs:
          learning_rate: The step size for the optimization.
          momentum: The momentum coefficient.
        '''

        self.learning_rate = learning_rate
        self.momentum = momentum
        self.velocity_filters = None
        self.velocity_biases = None

    def update(self, fitlers, biases, filters_grads, biases_grads):
        '''
        After calculating the gradients in the backpass, this function
        will update the parameters using Nesterov Accelerated Gradient.
        Inputs:
          filters: The current filters for the convLayer
          biases: The biases for the convLayer
          filters_grads: The gradients of the filters
          biases_grads: The gradients of the biases
        Output:
          Updated filters and biases.
        '''

        #Initalize the filter and biases velocity
        if self.velocity_filters is None:
            self.velocity_filters = np.zeros_like(self.filters)
        if self. velocity_biases is None:
            self.velocity_biases = np.zeros_like(self.biases)

        #lookahead for the filters and biases
        lookahead_filters = self.filters - self.momentum * self.velocity_filters
        lookahead_biases = self.biases - self.momentum * self.velocity_biases

        #update velocities
        self.velocity_filters = self.momentum * self.velocity_filters - self.learning_rate * filters_grads
        self.velocity_biases = self.momentum * self.velocity_biases - self.learning_rate * biases_grads

        #update parameters
        updated_filters = lookahead_filters + self.velocity_filters + self.momentum * (self.velocity_filters - self.momentum * self.velocity_filters)
        updated_biases = lookahead_biases + self.velocity_biases + self.momentum * (self.velocity_biases - self.momentum * self.velocity_biases)

        return updated_filters, updated_biases

##**Activation Function**

This might honestly end up being the easiest thing we've implemented all throughout this cookbook. After each convolotional layer, we need to add something called an fucntion function. What is that? It's just a mathematical function that tells neurons how to respond to an input. And why do we need it? A few reasons actually:
- The most important reason may be to introduce non-linerity in your network. This is the most important reason because non-linearity is needed to learn complex patterns in data (such as is the norm in the real-world).
- They transform the input signal into an output that can be passed to whatever layer comes next
- They choose which neurons to activate in the network and which ones to leave 'dead'.

They are so many activation functions out there like:
1. Sigmoid
2. Tanh
3. Rectified Linear Unit (ReLU)
4. Leaky ReLU
5. Paremetric ReLU (PReLU)
6. Exponential Linear Unit (ELU)
7. Maxout

In this project, we will be using PReLU, but for explanation, we will start from ReLU.

###Rectified Linear Unit
Ever heard of the phrase, "beauty in simplicity". Or maybe Occam's razor that states that among competing hypothesis, the simplest is the best. Or maybe the golden rule of computer science, K.I.S.S (Keep It Simple Stupid). ReLU is the embodiment of all these. It is the simplest activation function that you could implement and the most efficient computationally since it only does ONE simple mathematical operation: finding the maximum of 2 inputs. Just that. In fact the formula for it is just:
>$$f(x) = max(0,x)$$
> In code:
> ```
return np.max(0, x)
>```

Compared to the predeseccors of ReLU (Sigmoid and Tanh), this activation thrives in computational cost. It also solves a major problem that they had; saturation of neurons in the positive region. This means that the neuron's output are close to the asymptotic ends of a bounded activation function. I will not getinto all this in this notebook cause I'll have to explain other activations for that.

As much as it does solve the saturation problem, it does have 2 unique problems of it's own:
1. Not zero-centred. (Can be solved by sero-centering your data in the preprocessing step)
2. Kills the gradient in half the graph. Since ReLU compares an input value (logit) with 0, it means any negative input value will be reduced to 0. This reduction will lead to the killing of some neurons.

Often, this is ok, but to get just a little bit more efficieny from our NN, we have an activation function that attempts to solve the 2nd problem called Leaky ReLU.

###Leaky ReLU
Leaky ReLU is just an improvement of ReLU with only one change. Instead of comparing the inputs with $0$ we compare it with itself scaled by some very small positive value like $0.01$.
> $$f(x) = max(0.01, x)$$ In code
>```
return np.max((0.01*x), x)
>```

Quite simple, really. A very small change that solves a relatively huge problem. With this implementation, the negative side does not die, but rather have a small, but significant ouptut.

But we can take it a step further. The $0.01$ is just a constant in this case. However, we don't know if that's the best value. What if we could learn it like we do with all other paramters to find the optimal scalar value. Well, that gives us our activation function PReLU.

###Parametric ReLU
See why I had to start from ReLU to get here? Like I said, all we need to do is change something small to get a huge change. In this case, we get a learnable activation function.
> $$f(x) = max(αx,x)$$
where:
- α is a learnable parameter that can't be learned using backprop

To make a bit easier to implement in code, let's rewrite it a bit. (Be aware that all other forms of ReLU could be rewritten as follows, but we're only doing it with this one cause it's what we'll be implementing and makes backprop easier).
> $$PReLU(x) = f(x) =
\left\{
\begin{array}{11}
x, & \text{if }\quad x > 0 \\
α⋅x, & \text{otherwise}
\end{array}
\right\}$$
> The forward pass in code:
>```
return np.where(x > 0, x, self.alpha * x)
>```
> The back pass in code:
```
dx = np.where(self.input > 0, dy, self.alpha*dy )
self.alpha_grad = np.sum(dy * self.input * (self.input <= 0))
```
And that's it, honestly. Like I said, it's the simplest thing so far and we can use it for both the conv2D layer and FC layer.


####Actual PReLU implementation

In [ ]:
class PReLU():
  def __init__(self, alpha=0.01):
    '''
      This function initializes the PReLU activation function. Seriously, initialize this stuff, makes life so much easier.
    '''
    self.alpha = alpha
    self.alpha_grad = None

  def forward(self, prev_layer_input):
    '''
      This function does the forward pass of the PReLU activation function.
      Input:
        prev_layer_input: The input from the previous layer, be it the conv2D or the FC layer.
      Output:
        The output of the PReLU activation function.
    '''
    #storing the input with self to use in the backprop fun
    self.input = prev_layer_input

    return np.where(prev_layer_input > 0, prev_layer_input, self.alpha * prev_layer_input)

  def backward(self, dy):
    '''
      This function does the backward pass of the PReLU activation function.
      Input:
        dy: The gradient of the loss function with respect to the output of the PReLU activation function.
      Output:
        dx: The gradient of the loss function with respect to the input of the PReLU activation function.
    '''
    # Gradient of the activation with respect to the input
    dx = np.where(self.input > 0, dy, self.alpha * dy)

    # Gradient of alpha: sum of the gradients where input <= 0
    self.alpha_grad = np.sum(dy * self.input * (self.input <= 0))

    return dx

##**Pooling Layer**

To the second simplest concept in this whole thing. Pooling is an operation whose sole purpose is to reduce the dimensionality of an input so as to avoid the curse of dimensionality. Basically, the pooling layer makes the network invariant to the small translations in the input.
> The **curse of dimensionality** is a problem that arose from standard ML where the dimensions of the input are so large that it becomes computationally expensive to handle all the features. In standard ML, features are reduced using methods such as Principal Component Analysis (PCA).

In computer vision, we don't use PCA to reduce dimensionality since our features are not vectors of integers. They are, in fact, matrices. We would like to retain the spatial structure of the input features (filters) while reducing the spatial dimensionality. Enter pooling. Pooling works by simply sliding a filter (often a $2$x$2$ or $3$x$3$ matrix) with no values across a feature map (at a given stride) and identifying the most important/dominant feature weights in the covered region.

There are 2 major types of pooling:
1. Max Pooling: In this type of pooling, the filter picks the largest feature in a given covered region.
2. Average Pooling: For this, the filter does the average of the weights in the covered region and returns it as it's output.

For this project, we'll be using max pooling since it's the most common type of pooling used in CNNs all over. Before that, this layer has only one formula that is used to calculate the output structure size. It's really simple and familiar to the formula we saw with the Conv2D layer for calculating the output size.

Some notations:
- $H_{in}$ is the input height.
- $W_{in}$ is the input width.
- $H_{out}$ is the output height.
- $W_{out}$ is the output width.
- $K_h$ is the height of the pooling window (filter size).
- $K_w$ is the width of the pooling window.
- $S_v$ is the vertical stride (how much the window moves down).
- $S_h$ is the horizontal stride (how much the window moves right).
- $P_v$ is the vertical padding (if any).
- $P_h$ is the horizontal padding (if any).

>$$H_{out} = \left(\frac{H_n - K_h + 2P_v}{S_v}\right) +1$$
>$$W_{out} = \left(\frac{W_n - K_w + 2P_h}{S_h}\right) +1$$

With this, we can confirm whether we're getting the right dimensions for the output

>**NOTE:** The depth of the structure is ALWAYS retained. That should be obvious by now.

The backward pass for the pooling layer works in the exact opposite manner as the forward pass. In the backpass, we 'rebuild' our feature map with only the max element (from the previous layer) and 0's in place of other elements that we don't know off.

###Actual Pooling Layer

In [ ]:
class MaxPool2D():
  def __init__(self, pool_size, stride, padding):
    self.pool_size = pool_size
    self.stride = stride
    self.padding = padding
    self.input_shape = None
    self.input_map = None

  def forward(self, feature_map):
    '''
    This function is meant to act as the pooling layer after a Conv2D layer
    Inputs:
      feature_map: This is the output of the Conv2D layer
      pool_size: This is the size of the pooling filter
      stride: This the steps to be taken by the filter
      padding: This is the amount of zero padding to be added
    Output:
      output: This is the input structure with reduced spatial dimanesions of the pooling layer
    '''
    #account for any padding that may be added

    self.input_shape = feature_map.shape
    self.input_map = feature_map

    if self.padding > 0:
          feature_map = np.pad(feature_map,
                        ((0,0), (0,0), (self.padding, self.padding), (self.padding, self.padding)),
                        mode='constant',
                        constant_values=0)

    #Get the shapes for the input and the pooling filter
    batch_size, channels, H_in, W_in = feature_map.shape
    pool_h, pool_w = self.pool_size

    #calculate the output size
    W_out = (W_in - pool_w) // self.stride + 1
    H_out = (H_in - pool_h) // self.stride + 1

    H_out = max(1, H_out)
    W_out = max(1, W_out)

    self.max_indices = np.zeros_like(feature_map)

    #initialize the output
    output_map = np.zeros((batch_size, channels, H_out, W_out))

    #create the window and sliding
    for b in range(batch_size):
      for c in range(channels):
        for i in range(H_out):
          for j in range(W_out):
            #slide the window
            window = feature_map[b, c, i*self.stride : i*self.stride+pool_h,
                                j*self.stride : j*self.stride+pool_w]
            #perform max pooling
            output_map[b, c, i, j] = np.max(window)
            max_idx = np.unravel_index(np.argmax(window), window.shape)
            self.max_indices[b, c, i+max_idx[0], j+max_idx[1]] = 1


    return output_map

  def backward(self, dL_dOutput):
    '''
    This function is meant to propagate the feature map with repect to the input.
    Basically give us our gradient the same shape as the input
    Input:
      dL_dOutput: Gradients passed in the backprop
    '''
    dL_dInput = np.zeros_like(self.input_map)
    batch_size, channels, out_height, out_width = dL_dOutput.shape

    for b in range(batch_size):
        for c in range(channels):
            for i in range(out_height):
                for j in range(out_width):
                    # Find the corresponding window in the input
                    window_start_i = i * self.stride
                    window_start_j = j * self.stride
                    window_end_i = window_start_i + self.pool_size[0]
                    window_end_j = window_start_j + self.pool_size[0]

                    # Only the position of the max value gets the gradient
                    max_mask = self.max_indices[b, c, window_start_i:window_end_i, window_start_j:window_end_j]
                    dL_dInput[b, c, window_start_i:window_end_i, window_start_j:window_end_j] += dL_dOutput[b, c, i, j] * max_mask

    return dL_dInput

In [ ]:
#Beautiful code but does it work? Let's test and see

#create a test matrix that will work as our feature map
test_input = np.array([
     [1, 3, 2, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
])

# Apply pooling with 2x2 window, stride 1, and padding 1
output = MaxPool2D(test_input, pool_size=(2, 2), stride=1, padding=0)

print("Output after pooling:")
print(output)

Output after pooling:
[[ 6.  7.  8.]
 [10. 11. 12.]
 [14. 15. 16.]]


##**Flattening layer**

Remember how I said that the activation function is the simplest thing to implement here. Well, I sort of lied. The activation function was the simplest thing mathematically (so was the pooling layer since they're both just finding the max), but flattening may be the simplest thing completely.

So, what's the purpose of this layer? Well, after everything comes from the Conv2D layer and from the pooling layer, it has to go to a fully connected (dense) layer that can look at the features at a high level and do some reasoning. Now, the issue with this is that the FC layer expects a 1D vector as input. At this point, everything we've been doing gives a matrix as an ouput. We have to fix this, and we do this by 'flattening'. This layer has no math. It's all code, so let's get to it.

Basicaly, what we're doing is changing a tensor from a shape of $(D,H,W)$ into a 1D shape $(D⋅H\cdot W)$

In [ ]:
class Flatten:
  def __init__(self):
    '''
    An initialization function for the flatten layer.
    I think by now you should know what I'm going to say for this
    '''
    self.input_shape = None #to store the original shape for unflatening

  def flatten(self, input_tensor):
    '''
    Reshape the input tensor into a 1D vector.
    Input:
      input_tensor: The input tensor to be flattened.
    Output:
      The flattened tensor.
    '''
    self.input_shape = input_tensor.shape
    return input_tensor.reshape(-1)

  def unflatten(self, output_gradient):
    '''
    Reshape the output gradient from the FC layer into original shape
    Input:
      output_gradient: The output gradient from the FC layer.
    Output:
      The unflattened gradient.
    '''
    return output_gradient.reshape(self.input_shape)

In [ ]:
'''
To test whether our flattening layer is actually working, we'll assume a 3x4x4 output
from our pooling layer
'''

# Example output from a pooling layer
pool_output = np.random.rand(3, 4, 4)
print(pool_output)

#initialize the flattening layer
flatten_layer = Flatten()

# flattening pass
flattened_output = flatten_layer.flatten(pool_output)
print("Flat output:")
print(flattened_output)
print(f"Flat output shape: flattened_output.shape")  # Should print (48,)

# unflattening Pass
grad = np.random.rand(48)  # Example gradient from the dense layer
reshaped_grad = flatten_layer.unflatten(grad)
print("Reshaped grad:")
print(reshaped_grad)
print(f"Reshaped grad shape: {reshaped_grad.shape}")  # Should print (3, 4, 4)

[[[0.21748052 0.43818825 0.21033297 0.88470435]
  [0.39478603 0.89036283 0.76333468 0.18929517]
  [0.4741817  0.7623871  0.87488044 0.78947741]
  [0.18920341 0.11311195 0.96567126 0.35670834]]

 [[0.94550993 0.26021057 0.01891305 0.94974973]
  [0.52850599 0.21689779 0.30043856 0.16167789]
  [0.69240044 0.42257828 0.01353891 0.78531777]
  [0.34458198 0.09003186 0.65676747 0.30411179]]

 [[0.20505284 0.62440074 0.41007311 0.6165329 ]
  [0.95497529 0.69150772 0.12544577 0.04050557]
  [0.66568    0.83925949 0.58459667 0.78541371]
  [0.00942487 0.02456078 0.30717238 0.60611551]]]
Flat output:
[0.21748052 0.43818825 0.21033297 0.88470435 0.39478603 0.89036283
 0.76333468 0.18929517 0.4741817  0.7623871  0.87488044 0.78947741
 0.18920341 0.11311195 0.96567126 0.35670834 0.94550993 0.26021057
 0.01891305 0.94974973 0.52850599 0.21689779 0.30043856 0.16167789
 0.69240044 0.42257828 0.01353891 0.78531777 0.34458198 0.09003186
 0.65676747 0.30411179 0.20505284 0.62440074 0.41007311 0.6165329
 0.9

##**Fully Connected Layer (Dense)**

Seems we're back to some math, but this an easy concept. Before this, we discussed the flattening layer which takes in a 3-D tensor and 'flattens' it into a 1-D vector to serve as input to an FC layer. That's what we're about to talk about. So, first of all, what even is an FC layer? Think of it as the reasoning factor for your CNN. The Conv2D layer is meant to 'look' at the image and draw some features, but the FC layer is meant to use those extracted features to reason.

In this layer, every single input is connected to every single neuron, which means each neuron from the previous layer is connected to every single neuron of the next layer, hence the name fully connected layer. This means that every single neuron sees all inputs and makes some decision on it. These decisions are used to come up with some probabilty to make a final decision (to be discussed next). Also, this layer brings linearity back to the input. We will, in this project, choose to introduce non-linearity using an activation function, though.


###Forward pass


Mathematically, this is a very simple layer. It uses something from math that we all know about called 'the equation of a straight line'. Remember that?
>$$y = m\cdot x + c$$

That simple equation from high-school will be the foundation of this layer. For this layer we have a bit of a different formulation:
>$$z = W \cdot x + b$$
> where:
- $z$ is the output from a given neuron
- $W$ is the feature weight matrix of shape $m$ x $n$ where
  - $m$ is the number of output neurons (neurons in the next layer)
  - $n$ is the number of input neurons (input features)
- $x$ is the input vector of shape $n$ x $1$
- $b$ is the bias vector of shape $m$ x $1$

We can also choose to be a bit more strict and write it in terms of it's elements. It does look a lot more complicated, but it really isn't. The notaion is retained:
>$$z_i = \sum_{j=1}^n W_{ij} \cdot x_j + b_j$$

 If we are to implement an activation function, as we will, we'll have to add one more thing to our forward pass:
> $$a = f(z)$$
> where:
 - $a$ is the output from our activation function
 - $f(z)$ is our activation function
 - $z$ is the output from the linear function $W \cdot x +b$

In our case, we will be applying the PReLU that we wrote earlier.

And that's it. That's the whole forward pass for our FC layer.


###Backward pass

The backward pass for this layer is very, very similar to what we saw for the Conv2D layer. It's actually the exact same but much simpler since we're not dealing with matrices this time. For the backpass, we compute the gradient of the loss w.r.t the weights, biases, and inputs.

We'll start assuming that we won't add an activation function since we need to build from there. The formulas for the computations are as follows: (Remember in this cookbook, I'm keeping things simple, and will not do any derivations. You are free to derive them if you want, though)
1. Gradient w.r.t the weights:
> $$\frac{∂L}{∂W} = \frac{∂L}{∂z} \cdot x^T$$
>where:
  - $\frac{∂L}{∂z}$ is the derivative (gradient) of the loss w.r.t the layer output; recieved from the next layer
  - $x^T$ is the transposed input vector

2. Gradient w.r.t the biases:
> $$\frac{∂L}{∂b} = \frac{∂L}{∂z}$$
> If done using batch-processing, we can sum over all the examples since the bias is shared among all the examples. Therefore:
> $$\frac{∂L}{∂b} = \sum \frac{∂L}{∂z}$$

3. Gradient w.r.t the input:
>$$\frac{∂L}{∂x} = W^T \cdot \frac{∂z}{∂x}$$

But what if we had an activation function. Well, if we do, then things change a bit cause we need to account for the derivative of the activation function. That means that the function we are focusing on now will be $a = f(z)$.

1. Gradient w.r.t the weights:
> $$\frac{∂L}{∂W} = \left(\frac{∂L}{∂a} \odot f'(z)\right) \cdot x^T$$
>where:
  - $\frac{∂L}{∂a}$ is the derivative (gradient) of the loss w.r.t the activation function; recieved from the next layer
  - $x^T$ is the transposed input vector

2. Gradient w.r.t the biases:
> $$\frac{∂L}{∂b} = \sum \left(\frac{∂L}{∂a} \odot f'(z)\right) $$

3. Gradient w.r.t the input:
>$$\frac{∂L}{∂x} = W^T \cdot \left(\frac{∂L}{∂a} \odot f'(z)\right)$$

Note that the $⊙$ represent element-wise/pointwise multiplication.

A lot to digest, so let's put it all in code.

####Actual FC layer

In [ ]:
import numpy as np

class FullyConnectedLayer:
    def __init__(self, input_neurons_num, output_neurons_num, activation_function=None, activation_derivative=None):
        '''
        Initialize the fully connected layer. No comment!
        Input:
          input_neurons_num: Number of input neurons.
          output_neurons_num: Number of output neurons.
          activation_function: The activation function to apply.
          activation_derivative: The derivative of the activation function.
        '''
        self.input_size = input_neurons_num
        self.output_size = output_neurons_num
        self.activation_function = activation_function
        self.activation_derivative = activation_derivative

        # Initialize weights and biases with He initialization
        self.weights = np.random.randn(output_neurons_num, input_neurons_num) * np.sqrt(2 / input_neurons_num)
        self.biases = np.zeros((output_neurons_num, 1))

    def forward(self, input_data):
        '''
        Perform the forward pass through the FC layer.
        Input:
          input_data: Input data of shape (input_neurons_num, batch_size).
        Output:
          Output data after applying the activation function.
        '''

        self.input = input_data  # Store input for use in backward pass
        self.z = np.dot(self.weights, input_data) + self.biases
        if self.activation_function:
            return self.activation_function(self.z)
        else:
            return self.z

    def backward(self, output_gradient, learning_rate):
        '''
        Perform the backward pass through the FC layer.
        Input:
          output_gradient: Gradient of the loss w.r.t. the output (a) of this layer.
          learning_rate: Learning rate for weight updates.
        Output:
          dx Gradient of the loss w.r.t. the input (x) of this layer.
        '''

        # Compute gradient w.r.t. z
        if self.activation_derivative:
            dz = output_gradient * self.activation_derivative(self.z)  # Element-wise multiplication
        else:
            dz = output_gradient

        # Compute gradients
        dw = np.dot(dz, self.input.T) / self.input.shape[1]  # Weight gradient
        db = np.sum(dz, axis=1, keepdims=True) / self.input.shape[1]  # Bias gradient
        dx = np.dot(self.weights.T, dz)  # Input gradient

        # Update weights and biases
        self.weights -= learning_rate * dw
        self.biases -= learning_rate * db

        return dx


In [ ]:
#As usual, we need to test whether what we're doing makes sense or not
# create some test data
input_data = np.random.randn(128, 32)  # Input of shape (input_size, batch_size)
output_gradient = np.random.randn(64, 32)  # Dummy gradient from next layer

#initialize an instance of PReLU
actiavtion = PReLU()

#initialize a FC layer
fc_layer = FullyConnectedLayer(128, 64, activation_function=actiavtion.forward, activation_derivative=actiavtion.backward)

output = fc_layer.forward(input_data)
print(output.shape)
print(output)

# Backward pass
input_gradient = fc_layer.backward(output_gradient, learning_rate=0.01)
print(input_gradient.shape)
print(input_gradient)


(64, 32)
[[ 1.59732058e-01 -9.01600614e-03  2.29443749e+00 ... -2.81974620e-03
  -2.04431051e-02 -1.72507398e-02]
 [-3.19568989e-02  1.51436586e+00 -8.38444095e-03 ... -3.42938710e-03
  -4.88291027e-03  1.29221054e+00]
 [ 1.65197096e+00 -3.73681929e-04 -8.30110780e-03 ...  1.34533932e+00
  -1.67639117e-02 -1.07803963e-02]
 ...
 [-3.94632455e-03 -1.15067556e-04 -4.31311053e-04 ...  7.43291669e-01
  -7.19238499e-03 -1.92792588e-02]
 [ 3.21997497e-01  1.42102273e+00 -1.59259513e-02 ...  6.06669440e-01
   7.06317642e-01 -1.67623809e-02]
 [ 1.62881343e+00 -1.39328038e-02  1.87068450e+00 ...  3.52945141e+00
   1.90272467e+00  9.37999934e-01]]
(128, 32)
[[ 2.82525596e-02 -2.02839716e+00  7.76202472e-01 ...  1.83626004e+00
   5.58037100e-01  1.05835937e+00]
 [ 8.41143364e-01 -2.03476270e+00  1.17982482e+00 ...  9.55932375e-01
   4.89229950e-01 -6.80444188e-01]
 [ 4.47285094e-01  2.28066059e+00 -9.65314256e-01 ...  1.24617036e+00
   1.17733022e+00  3.09543604e-02]
 ...
 [ 1.44632294e-01 -9.8940

##**Softmax classifier**

Now after all that we've done that, we need to make decision and make some classifications. How do we do this? Well, if you're coming from basic ML, you already know we use probabilites for classification. But so far, we haven't dealt with probabilities, yet, but we have worked with non-linearity in the form of ReLU. Remember that was the function of the activation function in the first place. However, ReLU only does a max opeartion so we can't use it. We need another activation function at the last FC layer that will give us multiple probabilites for all our classes (10 classes for our MNIST dataset).

The most common activation function for this is called softmax. It's very similar to logistic regression but can be used for more than just binary classification. Obviously, the first thing to do is to write out the formula:
> $$a_j = \frac{e^{z_j}}{\sum_{k=1}^C e^{z_k}} = P(y = j | X)$$
> where:
- $e$ is euler's number $(≈2.71828...)$
- $C$ is the number of possible classes
- $z_j = W_j⋅x + b_j$ which is the output of the previous FC layer
- The denominator is the sum of the exponentials of all logits, ensuring that the output is normalized to 1

In code, this is very easy:
```
exp_x = np.exp(x - np.max(x, axis=0, keepdims=True))
return exp_x / np.sum(exp_x, axis=0, keepdims=True)
```
And that would be it for softmax. Our output will be the probability of all $C$ classes. In our case, $C=10$.

###Actual Softmax

Be aware the actual implementaion of Softmax will be in the activation file since it still an activation function.

In [ ]:
def softmax(x):
  '''
  This function is compute the class probabilites for a certain data point
  using softmax activation function
  Input:
    x: The input array of shape (batch_size, num_classes)
  Output:
    The class probabilites for the data point
  '''

  exp_x = np.exp(x - np.max(x, axis=0, keepdims=True))
  return exp_x / np.sum(exp_x, axis=0, keepdims=True)

##**Loss function**

Almost at the end of it all, now. After we've gotten the class probablities and made a decision by picking the one with the highest probability, we need to know whether our prediction is actually correct. This is a common thing even in standrad ML. To do this, we use a loss function that compares the predictions we've made against the true class. This loss function is meant to give our error rate/signal which is needed for optimization of our parameters. Remember the point of optimization is reducing a loss function to near 0.

There are a lot of loss functions developed over the years:
1. L1 loss (Mean Absolute Error)
2. L2 loss (Mean Squared Error)
3. Huber loss (Smooth Mean Absolute Error)
4. Log-Cosh loss
5. Quantile loss
6. Cross-entroypy loss/ log loss
7. Hinge loss

Actually, the way I've arranged them is specific. The first 5 are loss function developed majorly for regression problems, while the last 2 were developed for classiification problem. Our task is a classification problem and so we'll stick to one of those 2. More specifically, we'll pick cross-entropy because it's the most popular.

###Cross-entropy Loss

Like I said, this is the most common loss function used when it comes to classification tasks. The reason is that it was developed to be more generally, unlike Hinge loss which was originally made for SVMs (Support Vector Machine). It is used when the output of classification model is between 0 and 1. The closer the predicted probability is closer to the correct class, the lower the loss is. We derive the formulas from the likelihood function:
> $$L(θ|x) = f(x|θ) \quad \text{for } θ ∈ Θ$$

Now there are 2 different implementations of this function:
1. Binary cross-entropy: This is used when there are only 2 classes (binary classification)
2. Standard cross-entropy: This is used when there are more than 2 classes (multiclass classification)

I will write both implementations for the sake of fun:

Binary cross-entropy:
> $$L\left(\hat{y_i}, y_i\right) = J(θ) = -\frac{1}{m} \sum_{i = 1}^m \left(y_i⋅log(\hat{y_i})+(1-y_i)·log(1-\hat{y_i})\right)$$
> where:
- $L = J(Θ)$ is the loss function
- $θ$ are any parameters passing to the loss function such as weights and biases
- $m$ is the size of the training examples (data point)
- $y_i$ is the correct/true class of the $i$-th training example
- $\hat{y_i}$ is the predicted output by the model

>This is the easier formation of the formula. It's easier to implement but sometimes you may come across other formulations such as:
>> $$L\left(\hat{y_i}, y_i\right) = J(θ)
\left\{
\begin{array}{11}
-\text{log}(\hat{y_i}), & \text{if }\quad y_i = 1\\
-\text{log}(1 - \hat{y_1}), & \text{if }\quad y_i = 0
\end{array}
\right\}$$
Notations are retained.

Cross-entropy:
> $$L\left(\hat{y_i}, y_i\right) = -\frac{1}{m}\sum_{i = 1}^m y_i ·log(\hat{y_i})$$

We will implement both because I love my life and like to make things simpler (just in case I want to change the use case for this project).

###Actual loss function

In [ ]:
import numpy as np
class loss():
  def __init__(self, y_true, y_pred):
    '''
    Initialize the loss function.
    Input:
      y_true: The true labels.
      y_pred: The predicted labels.
    '''
    self.m = y_true.shape[0]
    self.y_true = y_true
    self.y_pred = y_pred

  def binary_cross_entropy(self):
    '''
    Compute the binary cross-entropy loss.
    Output:
      The binary cross-entropy loss.
    '''
    return -(1/self.m) * np.sum(self.y_true * np.log(self.y_pred) + (1 - self.y_true) * np.log(1 - self.y_pred))


  def cross_entropy(self):
    '''
    Compute the cross-entropy loss for multi-class classification.
    Output:
      The cross-entropy loss.
    '''
    batch_size = self.y_pred.shape[0]
    correct_class_probs = self.y_pred[np.arange(batch_size), self.y_true]
    log_probs = np.log(correct_class_probs + 1e-15)
    return -np.mean(log_probs)

In [ ]:
#as usual, test whether you've done bs
# Example usage with dummy data
logits = np.array([
    [1.0, 2.0, 3.0, 6.0],
    [0.5, 2.5, 2.0, 5.5],
    [0.3, 2.7, 4.5, 6.1]
])  # Logits of shape (3, 4) for a batch of 3 samples and 4 classes

labels = np.array([3, 1, 2])  # True labels (class indices)

# Step 1: Compute softmax probabilities
probabilities = softmax(logits)
print("Softmax Probabilities shape:")
print(probabilities.shape)

# Step 2: Compute cross-entropy loss
loss = loss(labels, probabilities)
loss = loss.cross_entropy()

print("Softmax Probabilities:\n", probabilities)
print("Cross-Entropy Loss:", loss)

Softmax Probabilities shape:
(3, 4)
Softmax Probabilities:
 [[0.47548496 0.21447841 0.17095278 0.36877214]
 [0.2883962  0.35361511 0.06289001 0.22367161]
 [0.23611884 0.43190648 0.76615721 0.40755625]]
Cross-Entropy Loss: 0.7678301433356479


##**Batch normalization**

At this point in time we have everything to implement a full CNN and a training loop. But we can still do some few other thing to make the network work better. One of the things we can do is by implementing batch normalization.

A very nice quote that I heard from a Standford course, "you want unit gaussian activations? Just make them so." This is a very good quote when you want to make your network better. Batch normalization does exactly what its name suggests and normalizes the output of a layer to ensure that they have a mean of 0 and a standard deviation of 1 during training. To do this, we need to compute the mean and variance for each feature in a mini-batch:
>Emprical mean ($μ$): $$μ = \frac{1}{m}\sum_{i=1}^m x_i$$
>Variance ($σ^2$): $$σ^2 = \frac{1}{m}\sum_{i=1}^m (x_i - μ)^2$$

To normalize, simply subtract the mean and divide the standard deviation:
> $$\hat{x}_i = \frac{x_i - μ}{\sqrt{σ^2 + ϵ}}$$
where:
 - $ϵ$ is some small positive number to avoid division by 0

After normalizing, the network has to 'squash' the range of the output as well. It does this by scaling ($γ$) and shifting ($β$) using the formula:
> $$y_k = γ_k⋅\hat{x_k} + β_k$$

All that is for the forward pass. For the back pass, we find the gradient of the loss w.r.t $\hat{x_k}$, $γ$ and $β$
> $$\frac{∂L}{∂γ} = \sum_{i =1}^k\frac{∂L}{∂y_i} ⋅ \hat{x_i}$$
> $$\frac{∂L}{∂β} = \sum_{i =1}^k\frac{∂L}{∂y_i}$$

As usual, I will dervivation to the reader (just implement the chain rule and substitute). Let's see the code.

###Actual batch nomalization

In [ ]:
class BatchNormalization:
  def __init__(self, input_size, epsilon=1e-5, momentum=0.9):
    '''
    I really don't know what else I can say about initilizations.
    Input:
      input_size: batch size
      epsilon: a small positive number to avoid division by 0
      momentum: momentum coefficient (friction) when using SGD with momentum
    '''
    self.gamma = np.ones((input_size, 1))
    self.beta = np.zeros((input_size, 1))
    self.epsilon = epsilon
    self.momentum = momentum
    self.running_mean = np.zeros((input_size, 1))
    self.running_var = np.zeros((input_size, 1))

  def forward(self, x, training=True):
    '''
    This is the forward pass for batch normalization
    Input:
      training: a boolean telling us whether the network should learn gamma and beta or not
    Output:
      The normalized output
    '''
    if training:
        self.mean = np.mean(x, axis=1, keepdims=True)
        self.var = np.var(x, axis=1, keepdims=True)
        self.x_hat = (x - self.mean) / np.sqrt(self.var + self.epsilon)

        # Update running estimates
        self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * self.mean
        self.running_var = self.momentum * self.running_var + (1 - self.momentum) * self.var
    else:
        # Use running estimates for inference
        self.x_hat = (x - self.running_mean) / np.sqrt(self.running_var + self.epsilon)

    self.out = self.gamma * self.x_hat + self.beta
    return self.out

  def backward(self, dout):
    '''
    This function is meant to 'undo' the effect of the normalization by backprop
    so as to compute the gradient of x
    Input:
      dout: the gradient of a later layer in the network

    Output:
      dx: the gradient of x backpropagated through the batch normalization
    '''
    m = dout.shape[1]

    # Gradients w.r.t. gamma and beta
    self.dgamma = np.sum(dout * self.x_hat, axis=1, keepdims=True)
    self.dbeta = np.sum(dout, axis=1, keepdims=True)

    # Backprop through normalization
    dx_hat = dout * self.gamma
    dvar = np.sum(dx_hat * (self.input - self.mean) * -0.5 * (self.var + self.epsilon)**-1.5, axis=1, keepdims=True)
    dmean = np.sum(dx_hat * -1 / np.sqrt(self.var + self.epsilon), axis=1, keepdims=True) + dvar * np.mean(-2 * (self.input - self.mean), axis=1, keepdims=True)

    dx = dx_hat / np.sqrt(self.var + self.epsilon) + dvar * 2 * (self.input - self.mean) / m + dmean / m
    return dx


##**Dropout regularization**

Well, this is another rather simple concept. From the title of this section, you can tell what this is. One of the major issues in AI/ML is the issue of overfitting where the model fits the training data so well that it can't generalise to new data. We are actually more likely to overfit with NNs considering how complex they are, meaning they are more likely to form even more complex relationships in the training data. That being said we have to deal with this overfitting issue.

In ML, we use techniques such as L1 and L2 regularization where we add a regularization term to the loss function. Unfortunately, this doesn't work as well with NNs. For this, we use a very simple concept called dropout. This is where we randomly 'kill' neurons but setting them to 0.

During training, we generate a binary mask $M$ with a keep probabilty $p$
> $$M \sim \text{Bernoulli}(p)$$
and then multiply the input by the mask
$$\text{output} = M ⊙ x$$

In [ ]:
class Dropout:
    def __init__(self, keep_prob=0.5):
        self.keep_prob = keep_prob

    def forward(self, x, training=True):
        if training:
            self.mask = (np.random.rand(*x.shape) < self.keep_prob) / self.keep_prob
            return x * self.mask
        else:
            return x  # During inference, no dropout is applied

    def backward(self, dout):
        return dout * self.mask


##**Evaluation Metrics**

Considering that we have everything that would be needed to build the CNN, we still need to know whether our CNN is actually performing well. Yes, the loss can tell us that, but that's majorly for the sake of learning in training. We want to evaluate the network after we have trained and on testing. To do this, there are a lot of ways that are common in both standard ML and here in DL. For that reason, I'll go straight to it.

For this project, I'll be using accuracy (the simplest evaluation metric ever) and a confusion matrix. The first is so easy that I'll just give the formula and you can see why:
> $$\text{Accuracy} = \frac{\text{Number of Correct Prediction}}{\text{Total Number of Predictions}}$$
Like I said, easy.

The confusion matrix may be in the relatively tricky one. It's difficult at all to understand. It's just a bit more verbose than accuracy. The point of a confusion matrix is to have a matrix of shape $C$ x $C$ where $C$ is the number of classes that shows the relation of between true class labels and predicted classes such that the leading diagonal represents the number of correctly predicted labels. To put it somewhat mathematically:
> $$\text{Confusion Matrix}[i,j] = \text{Number of times class }i \text{ is predicted as class }j$$

Can be easily and quickly implemented in code as follows:

###Actual evaluation metrics

In [ ]:
class evaluation():
  def __init__(self, y_true, y_pred):
    '''
    Initialize the evaluation metrics.
    Input:
      y_true: The true class labels.
      y_pred: The predicted labels by our network.
    '''
    self.y_true = y_true
    self.y_pred = y_pred

  def accuracy(self):
    '''
    Compute the accuracy of the model.
    Output:
      The accuracy of the model.
    '''
    return np.mean(self.y_true == self.y_pred)

  def confusion_matrix(self, num_classes):
    '''
    Create a confusion matrix to show the relationship between classes.
    Output:
      The confusion matrix.
    '''
    conf_matrix = np.zeros((num_classes, num_classes), dtype=int)
    for true_label, pred_label in zip(self.y_true, self.y_pred):
        conf_matrix[true_label, pred_label] += 1
    return conf_matrix

##**Let's put it together now**

Now, that we've implemented everything that we need to implement from scratch, we can start using them to build our fully functional CNN. We'll do this before implementing the training loop, so we can have the network architecture/design first.

So our architecture will look like this:
> Conv2D layer $⟶$ MaxPool2D layer $⟶$ Conv2D layer $⟶$ MaxPool2D layer $⟶$ Flattening layer $⟶$ FC1 layer $⟶$ FC2 layer

To be a bit more specific:
1. Conv2D layers have the following specs:
 - Filters: 32
 - Filter shape: $3$ x $3$
 - Activation: PReLU

2. Pooling Layer:
 - Window size: $2$ x $2$

3. Fully Connected Layer 1 (FC1):
 - Neurons: 128
 - Activation: PReLU
 - Dropout: 0.5

4. Fully Connected Layer 2 (FC2):
 - Neurons: 10
 - Activation: Softmax

 I'll use a class-based approach to simplify my life a bit.

In [ ]:
class CNN():
  def __init__(self):
    self.conv1 = Conv2DLayer(input_shape=(1,28,28), filter_shape = (32,1,3,3),stride=2, padding=2)
    self.pool1 = MaxPool2D(pool_size=(2,2), stride=2, padding=0)
    self.conv2 = Conv2DLayer(input_shape=(32,14,14), filter_shape = (64,32,3,3),stride=2, padding=2)
    self.pool2 = MaxPool2D(pool_size=(2,2), stride=2, padding=0)
    self.flatten = Flatten()
    self.fc1 = FullyConnectedLayer(1024, 128, activation_function=PReLU().forward, activation_derivative=PReLU().backward)
    self.dropout = Dropout(keep_prob=0.5)
    self.fc2 = FullyConnectedLayer(128, 10, activation_function=softmax().softmax, activation_derivative=None)

  def forward(self, x):
    out = self.conv1.forward(X)
    out = PReLU.forward(out)
    out = self.pool1.forward(out)
    out = self.conv2.forward(out)
    out = PReLU.forward(out)
    out = self.pool2.forward(out)
    out = self.flatten.forward(out)
    out = self.fc1.forward(out)
    out = PReLU.forward(out)
    out = self.dropout.forward(out)
    out = self.fc2.forward(out)
    out = self.softmax.forward(out)
    return out

  def backward(self, d_loss, learning_rate):
    grad = self.softmax.backward(d_loss)
    grad = self.fc2.backward(grad, learning_rate)
    grad = self.dropout.backward(grad)
    grad = PReLU.backward(grad)
    grad = self.fc1.backward(grad, learning_rate)
    grad = self.flatten.backward(grad)
    grad = self.pool2.backward(grad)
    grad = PReLU.backward(grad)
    grad = self.conv2.backward(grad, learning_rate)
    grad = self.pool1.backward(grad)
    grad = PReLU.backward(grad)
    grad = self.conv1.backward(grad, learning_rate)
    return grad

In [ ]:
# Initialize the network
cnn = CNN()

# Training parameters
epochs = 10
batch_size = 64
learning_rate = 0.001

# Training loop
for epoch in range(epochs):
    permutation = np.random.permutation(len(X_train))
    X_train_shuffled = X_train[permutation]
    y_train_shuffled = y_train[permutation]

    epoch_loss = 0
    for i in range(0, len(X_train), batch_size):
        # Mini-batch
        X_batch = X_train_shuffled[i:i + batch_size]
        y_batch = y_train_shuffled[i:i + batch_size]

        # Forward pass
        predictions = cnn.forward(X_batch)

        # Compute loss and accuracy
        loss = cross_entropy_loss(predictions, y_batch)
        epoch_loss += loss

        # Backward pass
        d_loss = cross_entropy_loss_derivative(predictions, y_batch)
        cnn.backward(d_loss, learning_rate)

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(X_train)}")


Combining these into a single and proper CNN will be done in another notebook that is all code (MNIST_full). This cookbook was majorly for explanations on the concepts used to implement the CNN.